In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [415]:
class L_Hopfield:
    def __init__(self, L: int, N: int, lamb: float, H: float) -> None:
        """
        Initialize L-hopfield class.
        Parameters:
        - L (int): The number of Hopfield nets
        - N (int): Dimension of memories (as well as each network)
        Returns: 
        - None
        """
        self.L = L
        self.K = L # For simplicity, # Memories == # Networks
        self.N = N
        self.memories = self.rademacher(self.N, self.L) # shape: (N, L)
        self.input = np.sign(np.sum(self.memories, axis = 1)) # shape: (N, )
        self.h = self.input.copy() # external fields based on the paper
        self.states = np.array([self.input for _ in range(self.L)]) # shape: (L, N)

        self.lamb = lamb
        self.H = H
        

    def rademacher(self, dimension: int, samples: int) -> np.ndarray:
        """
        Generates an array of i.i.d. Rademacher-distributed random variables, where 
        each entry is independently sampled from {-1, +1} with equal probability.
    
        Parameters:
        - dimension (int): The number of rows (features) in the output matrix.
        - samples (int): The number of columns (samples) in the output matrix.
    
        Returns:
        - np.ndarray: A (dimension, samples) array with Rademacher-distributed entries.
        """
        return np.random.choice([-1, 1], size=(dimension, samples))

    def mattis_magnetization(self, a: int, mu: int) -> float:
        """
        Calculates Mattis magnetization between state of a_th network and mu_th memory. 
        In the case of binary ±1 vectors, Mattis magnetization is exactly the same as cosine similarity
        
        Parameters:
        a (int): Index of targetted network in the range [0, L - 1].
        mu (int): Index of targetted memory in the range [0, K - 1].

        Returns:
        - float: A number between [-1, 1], representing the alignment between two vectors.
        """
        return np.dot(self.states[a], self.memories[:, mu])/self.N

    def intra_layer_energy(self) -> float:
        """
        Calculates the intra_layer contribution to energy function.

        Returns:
        - float: Total intra_layer energy contribution
        """
        
        total = 0
        for a in range(self.L):
            for mu in range(self.K):
                total += self.mattis_magnetization(a, mu)**2
                
        total *= -self.N
        
        return total

    def inter_layer_energy(self, lamb: float) -> float:
        """
        Calculates the inter_layer contribution to energy function.

        Parameters:
        - lamb (float): Controlling the intensity of the inter-layer interaction strength (lambda in the paper).
        
        Returns:
        - float: Total inter_layer energy contribution
        """
        
        total = 0
        for a in range(self.L):
            for b in range(self.L):
                if (a == b):
                    continue
                for mu in range(self.K):
                    total += (self.mattis_magnetization(a, mu) * self.mattis_magnetization(b, mu))**2
        total *= lamb*self.N

        return total

    def external_field_energy(self, H: float) -> float:
        """
        Calculates the external field contribution to energy function.

        Parameters:
        - H (float): Controlling the intensity of the external field.
        
        Returns:
        - float: Total external field energy contribution
        """
        return -H * np.sum(self.states.dot(self.h))
                   

    def energy(self) -> float:
        """
        Calculates the total energy function.

        Parameters: 
        - lamb (float): Controlling the intensity of the inter-layer interaction strength (lambda in the paper).
        - H (float): Controlling the intensity of the external field.

        Returns: 
        - float: Total energy.
        """
        return self.intra_layer_energy() + self.inter_layer_energy(self.lamb) + self.external_field_energy(self.H)

    def update(self, epochs: int) -> None:
        
        for _ in range(epochs):
            for a in range(self.L):
                noise = np.atanh(self.rademacher(self.N, 1))
                for i in range(self.N):
                    h_tilda = 0
                    for b in range(self.L):
                        h_tilda += (1 if a == b else -self.lamb)* np.sum([self.memories[i, mu] * np.dot(self.states[b], self.memories[:, mu]) for mu in range(self.K)])
                    h_tilda *= 1/self.N
                    h_tilda += self.H * self.h[i]
                    self.states[a][i] = np.sign(h_tilda + 0.2*noise[i][0])
                    
                    
        

In [416]:
net = L_Hopfield(L = 3, N = 1000, lamb=20, H= 0.4)

In [417]:
net.energy()

np.float64(18424.604866559996)

In [418]:
net.states.dot(net.memories)

array([[508, 496, 484],
       [508, 496, 484],
       [508, 496, 484]])

In [419]:
net.update(10)

/var/folders/p4/fv5qp61911x80fx4bspznbv00000gn/T/ipykernel_2175/1827597226.py:120: RuntimeWarning: divide by zero encountered in arctanh
  noise = np.atanh(self.rademacher(self.N, 1))


In [420]:
net.energy()

np.float64(-30.056528640000003)

In [421]:
net.states.dot(net.memories)

array([[-12,  -4,  28],
       [ 30, -66,  14],
       [-28,  72, -16]])